In [1]:
import shap
import torch
import numpy 
import transformers
from transformers import AutoTokenizer,AutoModelForSequenceClassification, Trainer,TextClassificationPipeline
import numpy as np
import pandas as pd

In [2]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, problem_type="multi_label_classification")
num_labels=5
model = AutoModelForSequenceClassification.from_pretrained('C:\RMM\Medical4\multilabelmodel', num_labels=num_labels).to('cuda')
pipe=TextClassificationPipeline(model=model,tokenizer=tokenizer,device=0)

In [3]:
cor_x = np.load("multilabeltest.npy",allow_pickle=True)
cor_reviews = [review[1] for review in cor_x]
cor_labels = [review[0] for review in cor_x]

In [5]:
def mask_top_k(k, pred_label_no_mask, values, returned_tokens):
    """
    masked the k tokens that have the max shap values
    :param k: specify the largest k value
    :param values: shap values
    :param returned_tokens: a list of tokens
    :return: review, which is a str constructed from a list words
    """
    shap_values_0, shap_values_1,shap_values_2,shap_values_3,shap_values_4 = zip(*values)
    if pred_label_no_mask == 0:
        values = shap_values_0
    elif  pred_label_no_mask == 1:
        values = shap_values_1
    elif  pred_label_no_mask == 2:
        values = shap_values_2  
    elif  pred_label_no_mask == 3:
        values = shap_values_3  
    elif  pred_label_no_mask == 4:
        values = shap_values_4
    # print(values)
    values = np.array(values)
    # ids_top_k = np.argpartition(values, -k)[-k:]
    ids_top_k = (-values).argsort()[:k]
    for idx in ids_top_k:
        # print(idx)
        returned_tokens[idx] = "[UNK] "
    masked_review = "".join(returned_tokens)
    # print(masked_review)
    return masked_review

def predict_label(pipe, masked_review):
    """
    predict the label for the masked_review
    :param pipe: pipeline
    :param masked_review: string
    :return: 0 or 1, indicating the label
    """
    prediction = pipe([masked_review])
    labelstr=prediction[0]['label']
    if labelstr == 'LABEL_0':
        pred_label=0
    elif labelstr == 'LABEL_1':
        pred_label=1
    elif labelstr == 'LABEL_2':
        pred_label=2
    elif labelstr== 'LABEL_3':
        pred_label=3
    elif labelstr == 'LABEL_4':
        pred_label=4
    return pred_label

In [7]:
shap_values_list = []
token_data_list = []
top_k = [1, 5, 9, 13,17,21,25,29,33,37,31]
all_labels =[]
# use GPU
gpu_explainer = shap.Explainer(pipe, tokenizer)
i = 0
for review, label in zip(cor_reviews, cor_labels):
    label=label-1
    print(f"process {i}-th review")
    i += 1
    label4review =[]
    label4review.append(label)
    # to-do: truncate review if len(review)>80
    tokens = tokenizer.tokenize(review)
    if len(tokens) > 80:
        tokens_truncated = tokens[:80]
        review = " ".join(token for token in tokens_truncated)
    pred_label_no_mask = predict_label(pipe, review) # predicted label for review without mask
    label4review.append(pred_label_no_mask)
    shap_values = gpu_explainer([review])
    values = shap_values.values[0] # 2-dim ndarray
    returned_tokens = shap_values.data[0]
    for k in top_k:
        masked_review = mask_top_k(k, pred_label_no_mask, values, returned_tokens) # mask review by the shap values
        predicted_label= predict_label(pipe, masked_review)
        label4review.append(predicted_label)
    # label4review = [True_label, pred_label_without_mask, masked_label_1, masked_label_2, masked_review_3, masked_review_4]
    all_labels.append(label4review)

process 0-th review
process 1-th review
process 2-th review
process 3-th review
process 4-th review
process 5-th review
process 6-th review
process 7-th review
process 8-th review
process 9-th review
process 10-th review
process 11-th review
process 12-th review
process 13-th review
process 14-th review
process 15-th review
process 16-th review
process 17-th review
process 18-th review
process 19-th review
process 20-th review
process 21-th review
process 22-th review
process 23-th review
process 24-th review
process 25-th review
process 26-th review
process 27-th review
process 28-th review
process 29-th review
process 30-th review
process 31-th review
process 32-th review
process 33-th review
process 34-th review
process 35-th review
process 36-th review
process 37-th review
process 38-th review
process 39-th review
process 40-th review
process 41-th review
process 42-th review
process 43-th review
process 44-th review
process 45-th review
process 46-th review
process 47-th review
pr

Token indices sequence length is longer than the specified maximum sequence length for this model (597 > 512). Running this sequence through the model will result in indexing errors


process 52-th review
process 53-th review
process 54-th review
process 55-th review
process 56-th review
process 57-th review
process 58-th review
process 59-th review
process 60-th review
process 61-th review
process 62-th review
process 63-th review
process 64-th review
process 65-th review
process 66-th review
process 67-th review
process 68-th review
process 69-th review
process 70-th review
process 71-th review
process 72-th review
process 73-th review
process 74-th review
process 75-th review
process 76-th review
process 77-th review
process 78-th review
process 79-th review
process 80-th review
process 81-th review
process 82-th review
process 83-th review
process 84-th review
process 85-th review
process 86-th review
process 87-th review
process 88-th review
process 89-th review
process 90-th review
process 91-th review
process 92-th review
process 93-th review
process 94-th review
process 95-th review
process 96-th review
process 97-th review
process 98-th review
process 99-th

In [8]:
df = pd.DataFrame(np.array(all_labels))

In [9]:
filtered_df = df.loc[df[0]==df[1]]
len(filtered_df)

140

In [11]:
from sklearn.metrics import accuracy_score

In [13]:
print(accuracy_score(filtered_df[1], filtered_df[2]), accuracy_score(filtered_df[1], filtered_df[3]), accuracy_score(filtered_df[1], filtered_df[4]), accuracy_score(filtered_df[1], filtered_df[5]),accuracy_score(filtered_df[1], filtered_df[6]),accuracy_score(filtered_df[1], filtered_df[7]),accuracy_score(filtered_df[1], filtered_df[8]),accuracy_score(filtered_df[1], filtered_df[9]),accuracy_score(filtered_df[1], filtered_df[10]),accuracy_score(filtered_df[1], filtered_df[11]),accuracy_score(filtered_df[1], filtered_df[12]))

0.7857142857142857 0.5642857142857143 0.5071428571428571 0.4928571428571429 0.4857142857142857 0.4714285714285714 0.45 0.45 0.45 0.45714285714285713 0.45714285714285713
